In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [2]:
data = pd.read_csv('../data/diabetes.csv')
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


# PRE-PROCESSING

In [4]:
df=data.copy()
df.shape

(768, 9)

In [5]:
df=df.drop(['BloodPressure'],axis=1)

In [6]:
df

,Pregnancies,Glucose,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,35,0,33.6,0.627,50,1
1,1,85,29,0,26.6,0.351,31,0
2,8,183,0,0,23.3,0.672,32,1
3,1,89,23,94,28.1,0.167,21,0
4,0,137,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...
763,10,101,48,180,32.9,0.171,63,0
764,2,122,27,0,36.8,0.340,27,0
765,5,121,23,112,26.2,0.245,30,0
766,1,126,0,0,30.1,0.349,47,1


In [7]:
df[['Glucose', 'SkinThickness', 'Insulin', 'BMI']]=df[['Glucose', 'SkinThickness', 'Insulin', 'BMI']].replace(0,np.nan)

In [8]:
df.fillna(df.median(), inplace=True)


In [23]:
numerical_cols = df.columns[df.columns != 'Outcome']

scaler = MinMaxScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])


NameError: name 'MinMaxScaler' is not defined

In [25]:
x = df.drop('Outcome',axis=1)
y = df['Outcome']

In [27]:
x

,Pregnancies,Glucose,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age
0,6,148.0,35.0,125.0,33.6,0.627,50
1,1,85.0,29.0,125.0,26.6,0.351,31
2,8,183.0,29.0,125.0,23.3,0.672,32
3,1,89.0,23.0,94.0,28.1,0.167,21
4,0,137.0,35.0,168.0,43.1,2.288,33
...,...,...,...,...,...,...,...
763,10,101.0,48.0,180.0,32.9,0.171,63
764,2,122.0,27.0,125.0,36.8,0.340,27
765,5,121.0,23.0,112.0,26.2,0.245,30
766,1,126.0,29.0,125.0,30.1,0.349,47


In [29]:
y

0      1
1      0
2      1
3      0
4      1
      ..
763    0
764    0
765    0
766    1
767    0
Name: Outcome, Length: 768, dtype: int64

In [33]:
from sklearn.preprocessing import StandardScaler



scaler = StandardScaler()
x = scaler.fit_transform(x)


In [35]:
x


array([[ 0.63994726,  0.86604475,  0.67064253, ...,  0.16661938,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.20506583, -0.01230129, ..., -0.85219976,
        -0.36506078, -0.19067191],
       [ 1.23388019,  2.01666174, -0.01230129, ..., -1.33250021,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 , -0.02157407, -0.69524511, ..., -0.910418  ,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.14279979, -0.01230129, ..., -0.34279019,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.94206766,  0.21534665, ..., -0.29912651,
        -0.47378505, -0.87137393]])